In [1]:
import os
import datetime
import time
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
import shutil

import rasterio
from rasterio import plot
from shapely.geometry import MultiPolygon, shape, Point
from shapely_geojson import dumps

from pathlib import Path
from pprint import pprint
from zipfile import ZipFile

from planet import api
from planet.api import filters

to use your own api keys and parameters, copy paste the `parameters.py.dist` file in the same folder and remove the `.dist` extention. You can then replace the string with your own keys. only the .dist will be pushed to the dist git rep. 

In [2]:
from parameters import *

In [3]:
def save_thumb(metadata_df):
    """ From the metadata dataframe, save the thumbnail
    in the corresponding folder:
    
    Args:
        metadata_df (pd.DataFrame)
        
    Return:
        stores thumbnails in folder
    """
    session = requests.Session()
    session.auth = (PLANET_API_KEY, '')
    auth = session.auth
    
    for index, row in metadata_df.iterrows():
        url = row.thumbnail
        date = row.date
        item_type = row.item_type
        cloud_cover = row.cloud_cover
        id_ = row.id
        sample_id = row.sample_id
        
        thumb_name = f'it{item_type}_cc{cloud_cover}_y{date.year}m{date.month}_{id_}.jpg'
        
        thumb_path = os.path.join(os.getcwd(),'thumbs', 
                                  str(sample_id), 
                                  str(date.year))
        
        Path(thumb_path).mkdir(parents=True, exist_ok=True)

        r = requests.get(url, auth=auth, stream=True)
        if r.status_code == 200:
            with open(os.path.join(thumb_path, thumb_name), 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

In [4]:
def build_request(aoi_geom, start_date, stop_date, cloud_cover=100):
    """build a data api search request for PS imagery.
    
    Args:
        aoi_geom (geojson): 
        start_date (datetime.datetime)
        stop_date (datetime.datetime)
    
    Returns:
        Request
    """
    
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('cloud_cover', lte=cloud_cover),
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )
    
    # Skipping REScene because is not orthorrectified and 
    # cannot be clipped.
    
    return filters.build_search_request(query, [
        'PSScene3Band', 
        'PSScene4Band', 
        'PSOrthoTile',
        'REOrthoTile',])

In [5]:
# search the data api
def search_data_api(request, client, limit=100000):
    """ Search items from a given request.
    
    """
    result = client.quick_search(request)
    
    # this returns a generator
    return result.items_iter(limit=limit)

In [6]:
def get_items(id_name, request):
    """ Get items using the request with the given parameters
    
    Args:
        row (geopandas.DataFrame.row): 
            gpd.df.row with two columns: id(index) and geometry
        
        start_date:
        
    """
    
    items = list(search_data_api(request, client))
    
    return [id_name, items]

In [7]:
def get_dataframe(items):
    
    items_metadata = [(f['properties']['acquired'],
                     f['id'], 
                     f['properties']['item_type'],
                     f['_links']['thumbnail'],
                     f['_permissions'],
                     f['geometry'],
                     f['properties']['cloud_cover'],
                     f
                    ) for f in items[1]]
    
    # Store into dataframe
    df = pd.DataFrame(items_metadata)
    df[0] = pd.to_datetime(df[0])
    df.columns=[
        'date', 
        'id', 
        'item_type', 
        'thumbnail', 
        'permissions', 
        'footprint', 
        'cloud_cover', 
        'metadata'
    ]
    df['sample_id'] = items[0]
    df.sort_values(by=['date'], inplace=True)
    df.reset_index()
    
    return df

In [8]:
def add_cover_area(metadata_df, sample_df):
    
    for idx, row in metadata_df.iterrows():
        
        g1 = sample_df.at[row.sample_id, 'geometry'] # sample geometry
        g2 = shape(row.footprint) # footprint geometry
        metadata_df.at[idx, 'cover_perc'] = (g1.intersection(g2).area/g1.area)

In [9]:
def build_order_from_metadata(metadata_df, samples_df, sample_id):
    
    filtered_df = metadata_df[metadata_df.sample_id==sample_id]
    
    items_by_type = [(item_type, filtered_df[filtered_df.item_type == item_type].id.to_list())
              for item_type in filtered_df.item_type.unique()]
    
    products_bundles = {
        
        # Is not possible to ask for analytic_dn in PSScene3Band, so the next option is visual
        # for more info go to https://developers.planet.com/docs/orders/product-bundles-reference/
        'PSScene3Band': "analytic,visual",
        'PSScene4Band': "analytic_udm2,analytic_sr,analytic",
        'PSOrthoTile': "analytic_5b_udm2,analytic_5b,analytic_udm2,analytic",
        'REOrthoTile': "analytic",
    }

    products_order = [
        {
            "item_ids":v, 
            "item_type":k, 
            "product_bundle": products_bundles[k]
        } for k, v in items_by_type
    ]
    
    # clip to AOI
    aoi_geojson = json.loads(dumps(samples_df.at[sample_id, 'geometry']))
    tools = [{
        'clip': {
            'aoi': aoi_geojson
        }
    },]
    
    order_request = {
        'name': f'sample_{sample_id}',
        'products': products_order,
        'tools': tools,
        'delivery': {
            'single_archive': True,
            'archive_filename':'{{name}}_{{order_id}}.zip',
            'archive_type':'zip'
        },
            'notifications': {
                       'email': False
        },
    }
    return order_request

In [10]:
def filter_dataframe_items(dataframe, item_type_score, months_score):
    """Filter and score each item according to the season and item_type
    
    Return:
        Dataframe with only one selected image per year.
        
    """
    # Create a copy to avoid mutate the initial df
    df = dataframe.copy()
    
    item_count_per_year = dict(df.groupby(df.date.dt.year).size())
    
    for k_year in item_count_per_year.keys():
        
        # Filter only years with more than one image
        if item_count_per_year[k_year] > 1:

            for idx, row in metadata_df.iterrows():
                
                month = row.date.month

                df.at[idx, 'season_score'] = months_score[month]
                df.at[idx, 'item_score'] = item_type_score[row['item_type']]

                cloud_cover = row['cloud_cover']

                if cloud_cover == 0:
                    df.at[idx, 'cloud_score'] = 10
                elif cloud_cover < 0.005:
                    df.at[idx, 'cloud_score'] = 5
                else:
                    df.at[idx, 'cloud_score'] = 0
    
    df['total_score'] = df.season_score + \
                        df.item_score + \
                        df.cloud_score
    
    df = df.sort_values(by=['total_score', 'date'], ascending=False)
    df['year'] = df.date.dt.year
    df = df.drop_duplicates(subset=['year'], keep='first')
    df = df.sort_values(by=['date'], ascending=False)
    
    return df

In [11]:
def track_order(order_id, client, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        order_info = client.get_individual_order(order_id).get()
        state = order_info['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(10)

# 1. Search items
### Get the samples dataframe

From a geojson plots file, create a geo pandas dataframe to store the geometries and the id of each plot, it'll be used as a geometry filter and to calculate the % of area covered by the items.

In [ ]:
# #create a geoDataFrame object from a .txt file 
# if os.path.isfile(FILENAME):
#     df = pd.read_csv(FILENAME, sep=' ')
    
#     #filter only the `nb_rows` first rows
#     nb_rows = len(df)
#     filter_df  = df[df.index.isin(range(nb_rows))]
#     df = filter_df
    
#     #create the geodataframe 
#     pts = [Point(df.loc[i][FILE_LNG], df.loc[i][FILE_LAT]) for i in range(len(df))]
#     sample_gdf = gpd.GeoDataFrame(data={'geometry': pts}, index=df[FILE_ID], crs="EPSG:4326")
#     sample_gdf.index.names = ['id']
#     sample_gdf = sample_gdf.to_crs('ESRI:54009')
#     sample_gdf['geometry'] = sample_gdf['geometry'].buffer(BUFFER_SIZE, cap_style=3)
#     sample_gdf = sample_gdf.to_crs("EPSG:4326")

# sample_gdf

In [13]:
# Read the samples and select the first geometry for test purposes

samples_gdf = gpd.read_file('shp/samples.geojson')
samples_gdf.drop('lnd_s__', axis=1, inplace=True)
samples_gdf.set_index('id', drop=True, inplace=True)

### Connect to client

In [14]:
client = api.ClientV1(api_key=PLANET_API_KEY)

### Define filters

In [15]:
# define test data for the filter
start_date = datetime.datetime(2009, 1, 1)
stop_date = datetime.datetime(2020, 12, 31)
cloud_cover_lte = 0.01

## 1.1 Get items for individual samples ((optional))
### Get items and metadata using filters

In [16]:
# Define AOI, by selecting the first row of the samples geodataframe

row_number = 1
aoi_geometry = json.loads(dumps(samples_gdf.iloc[row_number].geometry))
sample_id = samples_gdf.iloc[row_number].name

In [17]:
request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)
items = get_items(sample_id, request)

# Transform items into a pandas dataframe with useful columns
metadata_df = get_dataframe(items)

### Calculate percentage of covered area

Calculate the percentage of covered area from the sample area with the item footprint

In [18]:
# Mutate metadata_df and add the percentage of cover area
add_cover_area(metadata_df, samples_gdf)

In [19]:
# Remove items that are under 100% of covered area
metadata_df = metadata_df[metadata_df.cover_perc == 1]

In [37]:
metadata_df[metadata_df.date.dt.year == 2019].iloc[5].metadata

{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20190102_101202_1014',
  'assets': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20190102_101202_1014/assets/',
  'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20190102_101202_1014/thumb'},
 '_permissions': ['assets.analytic:download',
  'assets.analytic_dn:download',
  'assets.analytic_dn_xml:download',
  'assets.analytic_sr:download',
  'assets.analytic_xml:download',
  'assets.basic_analytic:download',
  'assets.basic_analytic_dn:download',
  'assets.basic_analytic_dn_nitf:download',
  'assets.basic_analytic_dn_rpc:download',
  'assets.basic_analytic_dn_rpc_nitf:download',
  'assets.basic_analytic_dn_xml:download',
  'assets.basic_analytic_dn_xml_nitf:download',
  'assets.basic_analytic_nitf:download',
  'assets.basic_analytic_rpc:download',
  'assets.basic_analytic_rpc_nitf:download',
  'assets.basic_analytic_xml:download',
  'assets.basic_analytic_xml_n

### Score items


In [38]:
# item_type_score
item_type_score = {
    'PSScene4Band':8, 
    'PSScene3Band':8, 
    'PSOrthoTile':10,
    'REOrthoTile':0,
    'SkySatScene':0,
}

# season score
months_score = {
    1: 7, 7:0,
    2: 7, 8:0,
    3: 5, 9:0,
    4: 0, 10:5,
    5: 0, 11:10,
    6: 0, 12:10,
}

# Clear

In [39]:
selected_items = filter_dataframe_items(metadata_df, item_type_score, months_score)

In [40]:
selected_items

,date,id,item_type,thumbnail,permissions,footprint,cloud_cover,metadata,sample_id,cover_perc,season_score,item_score,cloud_score,total_score,year
57,2020-02-07 09:13:49.061312+00:00,3113151_3042416_2020-02-07_0f49,PSOrthoTile,https://tiles.planet.com/data/v1/item-types/PS...,"[assets.analytic:download, assets.analytic_dn:...","{'coordinates': [[[-2.614214096939828, 7.16036...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,7.0,10.0,10.0,27.0,2020
113,2019-11-02 10:16:47.534975+00:00,2803478_3042416_2019-11-02_1010,PSOrthoTile,https://tiles.planet.com/data/v1/item-types/PS...,"[assets.analytic:download, assets.analytic_dn:...","{'coordinates': [[[-2.676484261420925, 7.16041...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,10.0,10.0,30.0,2019
226,2018-12-31 10:10:58.187374+00:00,1981292_3042416_2018-12-31_1027,PSOrthoTile,https://tiles.planet.com/data/v1/item-types/PS...,"[assets.analytic:download, assets.analytic_dn:...","{'coordinates': [[[-2.607862625993444, 7.16036...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,10.0,10.0,30.0,2018
358,2017-12-30 11:10:29.356735+00:00,1025716_3042416_2017-12-30_0f32,PSOrthoTile,https://tiles.planet.com/data/v1/item-types/PS...,"[assets.analytic:download, assets.analytic_dn:...","{'coordinates': [[[-2.670433197884167, 7.16041...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,10.0,10.0,30.0,2017
406,2016-11-24 09:49:59.236132+00:00,20161124_094959_0e0e,PSScene4Band,https://tiles.planet.com/data/v1/item-types/PS...,"[assets.analytic:download, assets.analytic_dn:...","{'coordinates': [[[-2.639525456096517, 7.15276...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,8.0,10.0,28.0,2016
412,2015-12-11 13:41:31.135851+00:00,20151211_134131_0c18,PSScene3Band,https://tiles.planet.com/data/v1/item-types/PS...,"[assets.analytic_dn:download, assets.analytic_...","{'coordinates': [[[-2.6419956830949847, 7.0479...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,8.0,10.0,28.0,2015
413,2014-01-21 11:39:24+00:00,20140121_113924_3042416_RapidEye-4,REOrthoTile,https://tiles.planet.com/data/v1/item-types/RE...,"[assets.analytic:download, assets.analytic_sr:...","{'coordinates': [[[-2.7870565, 7.3866711], [-2...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,7.0,0.0,10.0,17.0,2014
410,2013-12-20 11:33:57+00:00,20131220_113357_3042416_RapidEye-1,REOrthoTile,https://tiles.planet.com/data/v1/item-types/RE...,"[assets.analytic:download, assets.analytic_sr:...","{'coordinates': [[[-2.7870565, 7.3866711], [-2...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,0.0,10.0,20.0,2013
388,2012-12-21 11:47:09+00:00,20121221_114709_3042416_RapidEye-3,REOrthoTile,https://tiles.planet.com/data/v1/item-types/RE...,"[assets.analytic:download, assets.analytic_sr:...","{'coordinates': [[[-2.7870565, 7.3866711], [-2...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,0.0,10.0,20.0,2012
386,2011-12-14 11:44:47+00:00,20111214_114447_3042416_RapidEye-2,REOrthoTile,https://tiles.planet.com/data/v1/item-types/RE...,"[assets.analytic:download, assets.analytic_sr:...","{'coordinates': [[[-2.7870565, 7.3866711], [-2...",0.0,{'_links': {'_self': 'https://api.planet.com/d...,8650_21398,1.0,10.0,0.0,10.0,20.0,2011


### ((Optional)): Export thumbnails
Create thumbnails from the selected items (dataframe) and store them into a structured folder

In [28]:
save_thumb(metadata_df[metadata_df.cloud_cover == 0.01])

## 1.2 Get items for all plots and store into a big df
### Loop over all plots
Loop over all plots and get the items.

In [ ]:
# Create a list of dataframes 

df_list = []
for index, row in samples_gdf.iterrows():
    
    aoi_geometry = json.loads(dumps(row.geometry))
    sample_id = row.name
    
    request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)
    items = get_items(sample_id, request)
    
    # Transform items into a pandas dataframe with useful columns
    metadata_df = get_dataframe(items)
    
    selected_items = filter_dataframe_items(metadata_df, item_type_score, months_score)
    
    df_list.append(selected_items)
    
    del metadata_df

In [ ]:
# Concatenate dataframes from the df_list
all_df = pd.concat(df_list)

# 2. Order assets

In [ ]:
# To create the order we need a dataframe with 
# filtered items, and a samples_gdf where is the geometry to clip 
# each item.

orders = []
for idx, row in samples_gdf.iterrows():

    order = build_order_from_metadata(all_df, samples_gdf, sample_id=idx)
    orders.append(order)

In [ ]:
orders[1]

### Request order
<font color='red'>The following lines will start the order in the planet server, after the order is created and is running, there is no way to stop it.</font>

In [ ]:
# order_info = client.create_order(orders[1]).get()

In [ ]:
order_id = order_info['id']
order_id

In [ ]:
orders = client.get_orders().get()

In [ ]:
track_order(order_id, client)

# 3. Download

In [ ]:
api_models = client.download_order(order_id)
api_models[0].get_body().write() # Write the iamge file